In [86]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [87]:
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore')
import missingno
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.metrics import accuracy_score

from matplotlib import font_manager, rc
font_path = "/content/drive/MyDrive/L1/Notebooks/dataset/malgun.ttf"
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

# 카테고리별로 고객 행 나눔


In [88]:
tdf = pd.read_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/full_data_with_label_FINAL.pkl')
tdf.cloth_cat.unique()

array(['cloth_men', 'cloth_casual', 'cloth_etc', 'cloth_business',
       'uncategorized', 'cloth_lingerie', 'cloth_sports', 'cloth_kids',
       'cloth_women'], dtype=object)

In [89]:
tdf.cloth_cat.value_counts()

cloth_casual      176011
uncategorized      69006
cloth_etc          31297
cloth_lingerie     30444
cloth_business     11427
cloth_men          10803
cloth_kids          6349
cloth_sports        1943
cloth_women          643
Name: cloth_cat, dtype: int64

In [90]:
tdf['14purchase'] =  tdf.apply(lambda x: x['PURCHAMOUNT'] if x['Year'] == 2014 
                                        else 0, axis=1)
tdf['15purchase'] =  tdf.apply(lambda x: x['PURCHAMOUNT'] if x['Year'] == 2015 
                                        else 0, axis=1)
tdf

,CUSTNO,spent_less_x,RECEIPTNO,PURCHDATE,PURCHTIME,AFFIL,CLASSCODEL,CLASSCODEM,CLASSCODES,STORENO,GENDER,AGEGROUP,RESIDENCE,PURCHAMOUNT,Year,AFFIL_CLASS,spent_less_y,AFFIL_CLASS_2,cloth_cat,hobby_cat,14purchase,15purchase
0,00001,1,02857006,2014-10-05,17,A,5,0506,A050601,012,M,60세이상,060,342400,2014,A5,1,A0506,cloth_men,uncategorized,342400,0
1,00001,1,02857007,2014-10-05,17,A,5,0506,A050601,012,M,60세이상,060,78000,2014,A5,1,A0506,cloth_men,uncategorized,78000,0
2,00001,1,02857020,2014-10-05,17,A,5,0501,A050113,012,M,60세이상,060,500000,2014,A5,1,A0501,cloth_men,uncategorized,500000,0
3,00001,1,02947490,2014-10-19,12,A,4,0402,A040222,012,M,60세이상,060,314640,2014,A4,1,A0402,cloth_casual,uncategorized,314640,0
4,00001,1,02947512,2014-10-19,13,A,5,0503,A050306,012,M,60세이상,060,406980,2014,A5,1,A0503,cloth_etc,uncategorized,406980,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337918,19373,1,02844688,2014-10-03,20,A,4,0402,A040228,042,F,25세~29세,100,69000,2014,A4,1,A0402,cloth_casual,uncategorized,69000,0
337919,19373,1,05366898,2015-10-25,14,A,4,0402,A040217,010,F,25세~29세,100,29000,2015,A4,1,A0402,cloth_casual,uncategorized,0,29000
337920,19373,1,05242502,2015-10-04,15,A,4,0402,A040217,042,F,25세~29세,100,69900,2015,A4,1,A0402,cloth_casual,uncategorized,0,69900
337921,19375,0,07268767,2014-11-14,22,B,33,3304,B330403,016,F,25세~29세,049,12800,2014,B33,0,B3304,cloth_lingerie,uncategorized,12800,0


In [91]:
tdf = tdf[['CUSTNO','cloth_cat','14purchase','15purchase']]
tdf.head()

,CUSTNO,cloth_cat,14purchase,15purchase
0,00001,cloth_men,342400,0
1,00001,cloth_men,78000,0
2,00001,cloth_men,500000,0
3,00001,cloth_casual,314640,0
4,00001,cloth_etc,406980,0


In [92]:
# 카테고리별로 고객 행 나눔 (cloth_men)
cloth_men = tdf[tdf.cloth_cat == 'cloth_men']
cloth_men = cloth_men.reset_index()
cloth_men_purch_diff = cloth_men.groupby('CUSTNO').sum()[['14purchase','15purchase']]

cloth_men_purch_diff['14purchase'] = \
cloth_men_purch_diff.apply(lambda x: round(x['14purchase']/cloth_men_purch_diff[['14purchase']].sum(),4) , axis=1)
cloth_men_purch_diff['15purchase'] = \
cloth_men_purch_diff.apply(lambda x: x['15purchase']/cloth_men_purch_diff[['15purchase']].sum() , axis=1)

cloth_men_purch_diff['cloth_men_purch_diff'] = \
cloth_men_purch_diff.apply(lambda x: x['15purchase'] -x['14purchase'] , axis=1)
cloth_men_purch_diff = cloth_men_purch_diff.reset_index()
cloth_men_purch_diff

,CUSTNO,14purchase,15purchase,cloth_men_purch_diff
0,00001,0.0009,0.000000,-0.000900
1,00004,0.0001,0.000072,-0.000028
2,00006,0.0000,0.000071,0.000071
3,00007,0.0001,0.000000,-0.000100
4,00011,0.0000,0.000545,0.000545
...,...,...,...,...
4147,19326,0.0000,0.000175,0.000175
4148,19336,0.0000,0.000979,0.000979
4149,19341,0.0000,0.000116,0.000116
4150,19362,0.0001,0.000203,0.000103


In [93]:
# 카테고리별로 고객 행 나눔 (cloth_kids)
cloth_kids = tdf[tdf.cloth_cat == 'cloth_kids']
cloth_kids = cloth_kids.reset_index()
cloth_kids_purch_diff = cloth_kids.groupby('CUSTNO').sum()[['14purchase','15purchase']]

cloth_kids_purch_diff['14purchase'] = \
cloth_kids_purch_diff.apply(lambda x: x['14purchase']/cloth_kids_purch_diff[['14purchase']].sum() , axis=1)
cloth_kids_purch_diff['15purchase'] = \
cloth_kids_purch_diff.apply(lambda x: x['15purchase']/cloth_kids_purch_diff[['15purchase']].sum() , axis=1)

cloth_kids_purch_diff['cloth_kids_purch_diff'] = \
cloth_kids_purch_diff.apply(lambda x: x['15purchase'] -x['14purchase'] , axis=1)
cloth_kids_purch_diff = cloth_kids_purch_diff.reset_index()
cloth_kids_purch_diff

,CUSTNO,14purchase,15purchase,cloth_kids_purch_diff
0,00047,0.000119,0.000000,-0.000119
1,00060,0.000180,0.000000,-0.000180
2,00109,0.000120,0.000000,-0.000120
3,00123,0.000852,0.000000,-0.000852
4,00131,0.000000,0.000258,0.000258
...,...,...,...,...
2700,19329,0.000745,0.000000,-0.000745
2701,19335,0.001498,0.000483,-0.001015
2702,19342,0.000000,0.000370,0.000370
2703,19347,0.000000,0.000187,0.000187


In [94]:
# 카테고리별로 고객 행 나눔 (cloth_casual)
cloth_casual = tdf[tdf.cloth_cat == 'cloth_casual']
cloth_casual = cloth_casual.reset_index()
cloth_casual_purch_diff = cloth_casual.groupby('CUSTNO').sum()[['14purchase','15purchase']]

cloth_casual_purch_diff['14purchase'] = \
cloth_casual_purch_diff.apply(lambda x: x['14purchase']/cloth_casual_purch_diff[['14purchase']].sum() , axis=1)
cloth_casual_purch_diff['15purchase'] = \
cloth_casual_purch_diff.apply(lambda x: x['15purchase']/cloth_casual_purch_diff[['15purchase']].sum() , axis=1)

cloth_casual_purch_diff['cloth_casual_purch_diff'] = \
cloth_casual_purch_diff.apply(lambda x: x['15purchase'] -x['14purchase'] , axis=1)
cloth_casual_purch_diff = cloth_casual_purch_diff.reset_index()
cloth_casual_purch_diff

,CUSTNO,14purchase,15purchase,cloth_casual_purch_diff
0,00001,0.000074,0.000211,0.000137
1,00002,0.000189,0.000279,0.000091
2,00004,0.000042,0.000010,-0.000032
3,00006,0.000026,0.000064,0.000038
4,00007,0.000090,0.000772,0.000683
...,...,...,...,...
14924,19365,0.000004,0.000033,0.000029
14925,19369,0.000003,0.000004,0.000001
14926,19372,0.000137,0.000067,-0.000071
14927,19373,0.000010,0.000006,-0.000004


In [95]:
# 카테고리별로 고객 행 나눔 (cloth_lingerie)
cloth_lingerie = tdf[tdf.cloth_cat == 'cloth_lingerie']
cloth_lingerie = cloth_lingerie.reset_index()
cloth_lingerie_purch_diff = cloth_lingerie.groupby('CUSTNO').sum()[['14purchase','15purchase']]

cloth_lingerie_purch_diff['14purchase'] = \
cloth_lingerie_purch_diff.apply(lambda x: x['14purchase']/cloth_lingerie_purch_diff[['14purchase']].sum() , axis=1)
cloth_lingerie_purch_diff['15purchase'] = \
cloth_lingerie_purch_diff.apply(lambda x: x['15purchase']/cloth_lingerie_purch_diff[['15purchase']].sum() , axis=1)

cloth_lingerie_purch_diff['cloth_lingerie_purch_diff'] = \
cloth_lingerie_purch_diff.apply(lambda x: x['15purchase'] -x['14purchase'] , axis=1)
cloth_lingerie_purch_diff = cloth_lingerie_purch_diff.reset_index()
cloth_lingerie_purch_diff

,CUSTNO,14purchase,15purchase,cloth_lingerie_purch_diff
0,00002,0.000593,0.002572,0.001979
1,00006,0.000000,0.000350,0.000350
2,00007,0.000000,0.000114,0.000114
3,00008,0.000402,0.000418,0.000017
4,00009,0.000000,0.000432,0.000432
...,...,...,...,...
9986,19365,0.000022,0.000093,0.000071
9987,19369,0.000135,0.000146,0.000011
9988,19372,0.000282,0.000000,-0.000282
9989,19373,0.000024,0.000000,-0.000024


In [96]:
# 카테고리별로 고객 행 나눔 (cloth_sports)
cloth_sports = tdf[tdf.cloth_cat == 'cloth_sports']
cloth_sports = cloth_sports.reset_index()
cloth_sports_purch_diff = cloth_sports.groupby('CUSTNO').sum()[['14purchase','15purchase']]

cloth_sports_purch_diff['14purchase'] = \
cloth_sports_purch_diff.apply(lambda x: x['14purchase']/cloth_sports_purch_diff[['14purchase']].sum() , axis=1)
cloth_sports_purch_diff['15purchase'] = \
cloth_sports_purch_diff.apply(lambda x: x['15purchase']/cloth_sports_purch_diff[['15purchase']].sum() , axis=1)

cloth_sports_purch_diff['cloth_sports_purch_diff'] = \
cloth_sports_purch_diff.apply(lambda x: x['15purchase'] -x['14purchase'] , axis=1)
cloth_sports_purch_diff = cloth_sports_purch_diff.reset_index()
cloth_sports_purch_diff

,CUSTNO,14purchase,15purchase,cloth_sports_purch_diff
0,00042,0.000171,0.000000,-0.000171
1,00066,0.000226,0.000000,-0.000226
2,00095,0.000000,0.000099,0.000099
3,00147,0.000307,0.092367,0.092059
4,00160,0.000216,0.000000,-0.000216
...,...,...,...,...
1308,19275,0.000000,0.000148,0.000148
1309,19293,0.000000,0.000297,0.000297
1310,19308,0.005286,0.000000,-0.005286
1311,19318,0.000000,0.000693,0.000693


In [97]:
# # 카테고리별로 고객 행 나눔 (cloth_women)
cloth_women = tdf[tdf.cloth_cat == 'cloth_women']
cloth_women = cloth_women.reset_index()
cloth_women_purch_diff = cloth_women.groupby('CUSTNO').sum()[['14purchase','15purchase']]

cloth_women_purch_diff['14purchase'] = \
cloth_women_purch_diff.apply(lambda x: x['14purchase']/cloth_women_purch_diff[['14purchase']].sum() , axis=1)
cloth_women_purch_diff['15purchase'] = \
cloth_women_purch_diff.apply(lambda x: x['15purchase']/cloth_women_purch_diff[['15purchase']].sum() , axis=1)

cloth_women_purch_diff['cloth_women_purch_diff'] = \
cloth_women_purch_diff.apply(lambda x: x['15purchase'] -x['14purchase'] , axis=1)
cloth_women_purch_diff = cloth_women_purch_diff.reset_index()
cloth_women_purch_diff

,CUSTNO,14purchase,15purchase,cloth_women_purch_diff
0,00072,0.001462,0.001468,0.000006
1,00170,0.000000,0.005830,0.005830
2,00215,0.000000,0.000839,0.000839
3,00274,0.000000,0.000419,0.000419
4,00347,0.000325,0.001258,0.000934
...,...,...,...,...
489,19274,0.002436,0.000000,-0.002436
490,19293,0.000000,0.004614,0.004614
491,19303,0.002436,0.000000,-0.002436
492,19318,0.003086,0.000000,-0.003086


In [98]:
# 카테고리별로 고객 행 나눔 (etc)
cloth_etc = tdf[tdf.cloth_cat == 'cloth_etc']
cloth_etc = cloth_etc.reset_index()
cloth_etc_purch_diff = cloth_etc.groupby('CUSTNO').sum()[['14purchase','15purchase']]

cloth_etc_purch_diff['14purchase'] = \
cloth_etc_purch_diff.apply(lambda x: x['14purchase']/cloth_etc_purch_diff[['14purchase']].sum() , axis=1)
cloth_etc_purch_diff['15purchase'] = \
cloth_etc_purch_diff.apply(lambda x: x['15purchase']/cloth_etc_purch_diff[['15purchase']].sum() , axis=1)

cloth_etc_purch_diff['cloth_etc_purch_diff'] = \
cloth_etc_purch_diff.apply(lambda x: x['15purchase'] -x['14purchase'] , axis=1)
cloth_etc_purch_diff = cloth_etc_purch_diff.reset_index()
cloth_etc_purch_diff

,CUSTNO,14purchase,15purchase,cloth_etc_purch_diff
0,00001,0.000164,0.000113,-0.000051
1,00002,0.000142,0.000000,-0.000142
2,00006,0.000024,0.000061,0.000037
3,00007,0.000813,0.000000,-0.000813
4,00009,0.000656,0.000000,-0.000656
...,...,...,...,...
8127,19353,0.000021,0.000000,-0.000021
8128,19362,0.000019,0.000107,0.000088
8129,19364,0.000045,0.000000,-0.000045
8130,19369,0.000010,0.000000,-0.000010


In [99]:
# 카테고리별로 고객 행 나눔 (cloth_business)
cloth_business = tdf[tdf.cloth_cat == 'cloth_business']
cloth_business = cloth_business.reset_index()
cloth_business_purch_diff = cloth_business.groupby('CUSTNO').sum()[['14purchase','15purchase']]

cloth_business_purch_diff['14purchase'] = \
cloth_business_purch_diff.apply(lambda x: x['14purchase']/cloth_business_purch_diff[['14purchase']].sum() , axis=1)
cloth_business_purch_diff['15purchase'] = \
cloth_business_purch_diff.apply(lambda x: x['15purchase']/cloth_business_purch_diff[['15purchase']].sum() , axis=1)

cloth_business_purch_diff['cloth_business_purch_diff'] = \
cloth_business_purch_diff.apply(lambda x: x['15purchase'] -x['14purchase'] , axis=1)
cloth_business_purch_diff = cloth_business_purch_diff.reset_index()
cloth_business_purch_diff

,CUSTNO,14purchase,15purchase,cloth_business_purch_diff
0,00001,0.000317,0.000214,-0.000102
1,00004,0.000000,0.000253,0.000253
2,00006,0.000011,0.000000,-0.000011
3,00007,0.000187,0.000000,-0.000187
4,00009,0.000000,0.000100,0.000100
...,...,...,...,...
3745,19331,0.000342,0.000000,-0.000342
3746,19344,0.000000,0.000010,0.000010
3747,19362,0.000033,0.000000,-0.000033
3748,19371,0.000083,0.000000,-0.000083


### hobby

In [100]:
tdf = pd.read_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/cloth_hobby_classified.pkl')
tdf = tdf[(tdf.hobby_cat != 'otherclass')]
tdf = tdf[['hobby_cat','14purchase','15purchase']]
tdf.hobby_cat.value_counts()

uncategorized    6849500
hobby_golf         19281
hobby_pet          14113
hobby_child        12768
hobby_media        10775
hobby_outdoor      10379
hobby_health        1580
hobby_instex        1128
hobby_sports         411
Name: hobby_cat, dtype: int64

In [101]:
# 카테고리별로 고객 행 나눔 (hobby_media)
hobby_media = tdf[tdf.hobby_cat == 'hobby_media'].reset_index()
hobby_media_purch_diff = hobby_media.groupby('CUSTNO').sum()[['14purchase','15purchase']]

hobby_media_purch_diff['14purchase'] = \
hobby_media_purch_diff.apply(lambda x: x['14purchase']/hobby_media_purch_diff[['14purchase']].sum() , axis=1)
hobby_media_purch_diff['15purchase'] = \
hobby_media_purch_diff.apply(lambda x: x['15purchase']/hobby_media_purch_diff[['15purchase']].sum() , axis=1)

hobby_media_purch_diff['hobby_media_purch_diff'] = \
hobby_media_purch_diff.apply(lambda x: x['15purchase'] -x['14purchase'] , axis=1)
hobby_media_purch_diff = hobby_media_purch_diff.reset_index()
hobby_media_purch_diff

,CUSTNO,14purchase,15purchase,hobby_media_purch_diff
0,00021,0.000944,0.000000,-0.000944
1,00060,0.000000,0.000074,0.000074
2,00068,0.000162,0.000000,-0.000162
3,00082,0.000075,0.000000,-0.000075
4,00086,0.000000,0.000013,0.000013
...,...,...,...,...
3816,19322,0.000106,0.000000,-0.000106
3817,19329,0.000077,0.000000,-0.000077
3818,19334,0.000346,0.000000,-0.000346
3819,19335,0.000662,0.000552,-0.000110


In [102]:
# 카테고리별로 고객 행 나눔 (hobby_child)
hobby_child = tdf[tdf.hobby_cat == 'hobby_child'].reset_index()
hobby_child_purch_diff = hobby_child.groupby('CUSTNO').sum()[['14purchase','15purchase']]

hobby_child_purch_diff['14purchase'] = \
hobby_child_purch_diff.apply(lambda x: x['14purchase']/hobby_child_purch_diff[['14purchase']].sum() , axis=1)
hobby_child_purch_diff['15purchase'] = \
hobby_child_purch_diff.apply(lambda x: x['15purchase']/hobby_child_purch_diff[['15purchase']].sum() , axis=1)

hobby_child_purch_diff['hobby_child_purch_diff'] = \
hobby_child_purch_diff.apply(lambda x: x['15purchase'] -x['14purchase'] , axis=1)
hobby_child_purch_diff = hobby_child_purch_diff.reset_index()
hobby_child_purch_diff

,CUSTNO,14purchase,15purchase,hobby_child_purch_diff
0,00021,0.000228,0.000000,-0.000228
1,00047,0.000000,0.000205,0.000205
2,00050,0.000248,0.000000,-0.000248
3,00092,0.000637,0.000104,-0.000533
4,00093,0.000000,0.000093,0.000093
...,...,...,...,...
3607,19334,0.000000,0.000511,0.000511
3608,19335,0.000252,0.001244,0.000992
3609,19347,0.000000,0.000345,0.000345
3610,19352,0.000000,0.000059,0.000059


In [103]:
# 카테고리별로 고객 행 나눔 (hobby_pet)
hobby_pet = tdf[tdf.hobby_cat == 'hobby_pet'].reset_index()
hobby_pet_purch_diff = hobby_pet.groupby('CUSTNO').sum()[['14purchase','15purchase']]

hobby_pet_purch_diff['14purchase'] = \
hobby_pet_purch_diff.apply(lambda x: x['14purchase']/hobby_pet_purch_diff[['14purchase']].sum() , axis=1)
hobby_pet_purch_diff['15purchase'] = \
hobby_pet_purch_diff.apply(lambda x: x['15purchase']/hobby_pet_purch_diff[['15purchase']].sum() , axis=1)

hobby_pet_purch_diff['hobby_pet_purch_diff'] = \
hobby_pet_purch_diff.apply(lambda x: x['15purchase'] -x['14purchase'] , axis=1)
hobby_pet_purch_diff = hobby_pet_purch_diff.reset_index()
hobby_pet_purch_diff

,CUSTNO,14purchase,15purchase,hobby_pet_purch_diff
0,00002,0.000360,0.000000,-0.000360
1,00014,0.000000,0.000152,0.000152
2,00021,0.000000,0.000486,0.000486
3,00032,0.000090,0.000109,0.000019
4,00065,0.000084,0.000000,-0.000084
...,...,...,...,...
3087,19320,0.000000,0.000376,0.000376
3088,19340,0.000216,0.000000,-0.000216
3089,19347,0.000048,0.000000,-0.000048
3090,19365,0.000333,0.000084,-0.000249


In [104]:
# 카테고리별로 고객 행 나눔 (hobby_outdoor)
hobby_outdoor = tdf[tdf.hobby_cat == 'hobby_outdoor'].reset_index()
hobby_outdoor_purch_diff = hobby_outdoor.groupby('CUSTNO').sum()[['14purchase','15purchase']]

hobby_outdoor_purch_diff['14purchase'] = \
hobby_outdoor_purch_diff.apply(lambda x: x['14purchase']/hobby_outdoor_purch_diff[['14purchase']].sum() , axis=1)
hobby_outdoor_purch_diff['15purchase'] = \
hobby_outdoor_purch_diff.apply(lambda x: x['15purchase']/hobby_outdoor_purch_diff[['15purchase']].sum() , axis=1)

hobby_outdoor_purch_diff['hobby_outdoor_purch_diff'] = \
hobby_outdoor_purch_diff.apply(lambda x: x['15purchase'] -x['14purchase'] , axis=1)
hobby_outdoor_purch_diff = hobby_outdoor_purch_diff.reset_index()
hobby_outdoor_purch_diff

,CUSTNO,14purchase,15purchase,hobby_outdoor_purch_diff
0,00010,0.000000,0.000055,0.000055
1,00021,0.000000,0.000645,0.000645
2,00024,0.000000,0.001832,0.001832
3,00038,0.000052,0.000000,-0.000052
4,00040,0.000000,0.000207,0.000207
...,...,...,...,...
4836,19335,0.000015,0.000631,0.000616
4837,19347,0.000021,0.000000,-0.000021
4838,19348,0.000000,0.000603,0.000603
4839,19361,0.000000,0.000703,0.000703


In [105]:
# 카테고리별로 고객 행 나눔 (hobby_sports)
hobby_sports = tdf[tdf.hobby_cat == 'hobby_sports'].reset_index()
hobby_sports_purch_diff = hobby_sports.groupby('CUSTNO').sum()[['14purchase','15purchase']]

hobby_sports_purch_diff['14purchase'] = \
hobby_sports_purch_diff.apply(lambda x: x['14purchase']/hobby_sports_purch_diff[['14purchase']].sum() , axis=1)
hobby_sports_purch_diff['15purchase'] = \
hobby_sports_purch_diff.apply(lambda x: x['15purchase']/hobby_sports_purch_diff[['15purchase']].sum() , axis=1)

hobby_sports_purch_diff['hobby_sports_purch_diff'] = \
hobby_sports_purch_diff.apply(lambda x: x['15purchase'] -x['14purchase'] , axis=1)
hobby_sports_purch_diff = hobby_sports_purch_diff.reset_index()
hobby_sports_purch_diff

,CUSTNO,14purchase,15purchase,hobby_sports_purch_diff
0,00042,0.002100,0.000000,-0.002100
1,00066,0.002786,0.000000,-0.002786
2,00160,0.002660,0.000000,-0.002660
3,00226,0.003917,0.000000,-0.003917
4,00347,0.004900,0.000000,-0.004900
...,...,...,...,...
303,18773,0.005572,0.000000,-0.005572
304,18821,0.013931,0.000000,-0.013931
305,18835,0.002800,0.000000,-0.002800
306,18952,0.000000,0.026288,0.026288


In [106]:
# 카테고리별로 고객 행 나눔 (hobby_health)
hobby_health = tdf[tdf.hobby_cat == 'hobby_health'].reset_index()
hobby_health_purch_diff = hobby_health.groupby('CUSTNO').sum()[['14purchase','15purchase']]

hobby_health_purch_diff['14purchase'] = \
hobby_health_purch_diff.apply(lambda x: x['14purchase']/hobby_health_purch_diff[['14purchase']].sum() , axis=1)
hobby_health_purch_diff['15purchase'] = \
hobby_health_purch_diff.apply(lambda x: x['15purchase']/hobby_health_purch_diff[['15purchase']].sum() , axis=1)

hobby_health_purch_diff['hobby_health_purch_diff'] = \
hobby_health_purch_diff.apply(lambda x: x['15purchase'] -x['14purchase'] , axis=1)
hobby_health_purch_diff = hobby_health_purch_diff.reset_index()
hobby_health_purch_diff

,CUSTNO,14purchase,15purchase,hobby_health_purch_diff
0,00021,0.000000,0.000646,0.000646
1,00110,0.000000,0.000448,0.000448
2,00129,0.002460,0.000000,-0.002460
3,00139,0.000000,0.001504,0.001504
4,00147,0.002736,0.000844,-0.001892
...,...,...,...,...
1118,19253,0.000541,0.000000,-0.000541
1119,19287,0.000472,0.000000,-0.000472
1120,19313,0.000492,0.000000,-0.000492
1121,19328,0.000000,0.000627,0.000627


In [107]:
# 카테고리별로 고객 행 나눔 (hobby_instex)
hobby_instex = tdf[tdf.hobby_cat == 'hobby_instex'].reset_index()
hobby_instex_purch_diff = hobby_instex.groupby('CUSTNO').sum()[['14purchase','15purchase']]

hobby_instex_purch_diff['14purchase'] = \
hobby_instex_purch_diff.apply(lambda x: x['14purchase']/hobby_instex_purch_diff[['14purchase']].sum() , axis=1)
hobby_instex_purch_diff['15purchase'] = \
hobby_instex_purch_diff.apply(lambda x: x['15purchase']/hobby_instex_purch_diff[['15purchase']].sum() , axis=1)

hobby_instex_purch_diff['hobby_instex_purch_diff'] = \
hobby_instex_purch_diff.apply(lambda x: x['15purchase'] -x['14purchase'] , axis=1)
hobby_instex_purch_diff = hobby_instex_purch_diff.reset_index()
hobby_instex_purch_diff

,CUSTNO,14purchase,15purchase,hobby_instex_purch_diff
0,00113,0.001272,0.000000,-0.001272
1,00126,0.001279,0.000000,-0.001279
2,00168,0.000483,0.000000,-0.000483
3,00185,0.001918,0.000000,-0.001918
4,00216,0.000000,0.004545,0.004545
...,...,...,...,...
898,19113,0.000000,0.001114,0.001114
899,19129,0.000000,0.000341,0.000341
900,19287,0.000355,0.000000,-0.000355
901,19335,0.000000,0.000773,0.000773


In [108]:
# 카테고리별로 고객 행 나눔 (hobby_golf)
hobby_golf = tdf[tdf.hobby_cat == 'hobby_golf'].reset_index()
hobby_golf_purch_diff = hobby_golf.groupby('CUSTNO').sum()[['14purchase','15purchase']]

hobby_golf_purch_diff['14purchase'] = \
hobby_golf_purch_diff.apply(lambda x: x['14purchase']/hobby_golf_purch_diff[['14purchase']].sum() , axis=1)
hobby_golf_purch_diff['15purchase'] = \
hobby_golf_purch_diff.apply(lambda x: x['15purchase']/hobby_golf_purch_diff[['15purchase']].sum() , axis=1)

hobby_golf_purch_diff['hobby_golf_purch_diff'] = \
hobby_golf_purch_diff.apply(lambda x: x['15purchase'] -x['14purchase'] , axis=1)
hobby_golf_purch_diff = hobby_golf_purch_diff.reset_index()
hobby_golf_purch_diff

,CUSTNO,14purchase,15purchase,hobby_golf_purch_diff
0,00002,0.000636,0.000168,-0.000468
1,00004,0.000075,0.000000,-0.000075
2,00006,0.000000,0.000012,0.000012
3,00007,0.000003,0.000000,-0.000003
4,00014,0.000022,0.000041,0.000019
...,...,...,...,...
3710,19315,0.000019,0.000000,-0.000019
3711,19326,0.000000,0.000068,0.000068
3712,19331,0.000133,0.000000,-0.000133
3713,19349,0.000017,0.000000,-0.000017


In [109]:
# clean_df = df.drop(['RECEIPTNO','PURCHDATE', 'PURCHTIME', 'AFFIL', 'CLASSCODEL',
#        'CLASSCODEM', 'CLASSCODES','STORENO', 'RESIDENCE', 'PURCHAMOUNT', 'Year', 'AFFIL_CLASS'], axis=1)
# clean_df.info()

In [110]:
clean_df = pd.read_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/custno_only.pkl')

In [111]:
temp = clean_df.copy()

- 원데이터에 별수 

In [112]:
temp1 = pd.merge(temp, 
                 cloth_men_purch_diff[['CUSTNO','cloth_men_purch_diff']], on='CUSTNO', how='left').fillna(0)
temp2 = pd.merge(temp1, 
                 cloth_kids_purch_diff[['CUSTNO','cloth_kids_purch_diff']], on='CUSTNO', how='left').fillna(0)
temp3 = pd.merge(temp2, 
                 cloth_casual_purch_diff[['CUSTNO','cloth_casual_purch_diff']], on='CUSTNO', how='left').fillna(0)
temp4 = pd.merge(temp3, 
                 cloth_lingerie_purch_diff[['CUSTNO','cloth_lingerie_purch_diff']], on='CUSTNO', how='left').fillna(0)
temp5 = pd.merge(temp4, 
                 cloth_sports_purch_diff[['CUSTNO','cloth_sports_purch_diff']], on='CUSTNO', how='left').fillna(0)
temp6 = pd.merge(temp5, 
                 cloth_etc_purch_diff[['CUSTNO','cloth_etc_purch_diff']], on='CUSTNO', how='left').fillna(0)
temp7 = pd.merge(temp6, 
                 cloth_business_purch_diff[['CUSTNO','cloth_business_purch_diff']], on='CUSTNO', how='left').fillna(0)

temp8 = pd.merge(temp7, 
                 hobby_media_purch_diff[['CUSTNO','hobby_media_purch_diff']], on='CUSTNO', how='left').fillna(0)
temp9 = pd.merge(temp8, 
                 hobby_child_purch_diff[['CUSTNO','hobby_child_purch_diff']], on='CUSTNO', how='left').fillna(0)
temp10 = pd.merge(temp9, 
                 hobby_pet_purch_diff[['CUSTNO','hobby_pet_purch_diff']], on='CUSTNO', how='left').fillna(0)
temp11 = pd.merge(temp10, 
                 hobby_outdoor_purch_diff[['CUSTNO','hobby_outdoor_purch_diff']], on='CUSTNO', how='left').fillna(0)
temp12 = pd.merge(temp11, 
                 hobby_sports_purch_diff[['CUSTNO','hobby_sports_purch_diff']], on='CUSTNO', how='left').fillna(0)
temp13 = pd.merge(temp12, 
                 hobby_health_purch_diff[['CUSTNO','hobby_health_purch_diff']], on='CUSTNO', how='left').fillna(0)
temp14 = pd.merge(temp13, 
                 hobby_instex_purch_diff[['CUSTNO','hobby_instex_purch_diff']], on='CUSTNO', how='left').fillna(0)
temp15 = pd.merge(temp14, 
                 cloth_women_purch_diff[['CUSTNO','cloth_women_purch_diff']], on='CUSTNO', how='left').fillna(0)
clean_df = pd.merge(temp15, 
                 hobby_golf_purch_diff[['CUSTNO','hobby_golf_purch_diff']], on='CUSTNO', how='left').fillna(0)

In [113]:
clean_df.groupby('CUSTNO').count()

,spent_less,cloth_men_purch_diff,cloth_kids_purch_diff,cloth_casual_purch_diff,cloth_lingerie_purch_diff,cloth_sports_purch_diff,cloth_etc_purch_diff,cloth_business_purch_diff,hobby_media_purch_diff,hobby_child_purch_diff,hobby_pet_purch_diff,hobby_outdoor_purch_diff,hobby_sports_purch_diff,hobby_health_purch_diff,hobby_instex_purch_diff,cloth_women_purch_diff,hobby_golf_purch_diff
CUSTNO,,,,,,,,,,,,,,,,,
00001,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
00002,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
00004,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
00006,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
00007,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19369,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
19371,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
19372,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [114]:
clean_df.to_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/X_cloth_hobby_var.pkl')